In [7]:
import imgaug as ia
from imgaug import augmenters as iaa
#from files import *
#from pascal_voc_writer import Writer
#import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
#import imgaug as ia
#from imgaug import augmenters as iaa
#from files import *

def read_train_dataset(img_dir):
    images = []
    annotations = []
    filenames = []

    for file in os.listdir(img_dir):
        filename = os.path.basename(file)
        #print(filename)
        filenames.append(filename)
        if 'jpg' in file.lower() or 'png' in file.lower():
            img = cv2.imread(img_dir + file, 1)
            H,W,_ = img.shape
            images.append(img)
            annotation_file = file.replace(file.split('.')[-1], 'txt')
            bounding_box_list, file_name = read_anntation(img_dir + annotation_file, W, H)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)
    #print(images)

    return images, annotations

def read_anntation(txt_file:str,W,H):

    bounding_box_list = []
    file_name = os.path.splitext(os.path.basename(txt_file))[0]+'.jpg'
    f = open(txt_file, 'r')
    
    while True:
        line = f.readline()
        if not line : break
        line = line.strip('\n').split(" ")
        label = line[0]
        to_float = lambda x:float(x)
        coord = list(map(to_float, line[1:]))
        box = coord * np.array([W,H,W,H])
        (centerX, centerY, width, height) = box.astype('int')
        
        x = int(centerX - (width / 2))
        y = int(centerY - (height/2))
        bounding_box_list.append([label, x,y,int(x+width),int(y+height)])
        

    f.close()
    return bounding_box_list, file_name

def yolo_convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[2])/2.0
    y = (box[1] + box[3])/2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    #print(x,y,w,h)
    return (x,y,w,h)

ia.seed(1)


img_dir = '/home/capstone7/data/img_and_txt/glass/'
save_dir = '/home/capstone7/data/aug_result/glass3/'
images, annotations= read_train_dataset(img_dir)

sometimes = lambda aug: iaa.Sometimes(0.5, aug)

for idx in range(len(images)):
    image = images[idx]
#     filename = filenames[idx]

    #txtfile = open(savd_dir+)
    
    boxes = annotations[idx][0]
    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)
    
    seq = iaa.Sequential([
        iaa.Sometimes(0.7, iaa.Add(-40)),
        iaa.Sometimes(0.7,iaa.Multiply((0.75, 1.25))),
        iaa.Sometimes(0.3,iaa.MotionBlur(k=5, angle=[-45,45])),
        iaa.Sometimes(0.5, iaa.Fliplr(0.5))
    ], random_order=True)
    
    seq_det = seq.to_deterministic()

    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    
    
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
#         print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
#             i,
#             before.x1, before.y1, before.x2, before.y2,
#             after.x1, after.y1, after.x2, after.y2)
#         )
    image_after = bbs_aug.draw_on_image(image_aug, thickness=5, color=[0, 0, 255])
#     cv2.imwrite("test.jpg", image_after)
    
    
    new_image_file = save_dir + 'augm_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    new_txt_file = open(save_dir+'augm_'+annotations[idx][1], 'w')

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        bb_box_ = [int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2)]
        yolo_style = yolo_convert((w,h), bb_box_)
        line = ("%s %f %f %f %f\n" % (boxes[i][0],yolo_style[0], yolo_style[1],yolo_style[2], yolo_style[3]))
        new_txt_file.write(line)

    
    new_txt_file.close()
    

print("done!!")



done!!


In [38]:
import os
import glob

path = '/home/capstone7/Downloads/Capstone_Greenbattle/yolo_data/yolo_dent_txt/'
save_dir = '/home/capstone7/Downloads/Capstone_Greenbattle/yolo_data/dent_aug/'
files = glob.glob('*')

for f in os.listdir(path):
    filename, fileext = os.path.splitext(f)
    
    if fileext == '.txt':
        os.rename(path+f, save_dir+'aug_'+f)

print("done")

done


In [1]:
#########glass augmentation

import imgaug as ia
from imgaug import augmenters as iaa
#from files import *
#from pascal_voc_writer import Writer
#import xml.etree.ElementTree as ET
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
#import imgaug as ia
#from imgaug import augmenters as iaa
#from files import *

def read_train_dataset(img_dir):
    images = []
    annotations = []
    filenames = []

    for file in os.listdir(img_dir):
        filename = os.path.basename(file)
        #print(filename)
        filenames.append(filename)
        if 'jpg' in file.lower() or 'png' in file.lower():
            img = cv2.imread(img_dir + file, 1)
            H,W,_ = img.shape
            images.append(img)
            annotation_file = file.replace(file.split('.')[-1], 'txt')
            bounding_box_list, file_name = read_anntation(img_dir + annotation_file, W, H)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)
    #print(images)

    return images, annotations

def read_anntation(txt_file:str,W,H):

    bounding_box_list = []
    file_name = os.path.splitext(os.path.basename(txt_file))[0]+'.jpg'
    f = open(txt_file, 'r')
    
    while True:
        line = f.readline()
        if not line : break
        line = line.strip('\n').split(" ")
        label = line[0]
        to_float = lambda x:float(x)
        coord = list(map(to_float, line[1:]))
        box = coord * np.array([W,H,W,H])
        (centerX, centerY, width, height) = box.astype('int')
        
        x = int(centerX - (width / 2))
        y = int(centerY - (height/2))
        bounding_box_list.append([label, x,y,int(x+width),int(y+height)])
        

    f.close()
    return bounding_box_list, file_name

def yolo_convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[2])/2.0
    y = (box[1] + box[3])/2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    #print(x,y,w,h)
    return (x,y,w,h)

ia.seed(1)


img_dir = '/home/capstone7/data/img_and_txt/glass/'
save_dir = '/home/capstone7/data/aug_result/glass2/'
images, annotations= read_train_dataset(img_dir)

sometimes = lambda aug: iaa.Sometimes(0.5, aug)

for idx in range(len(images)):
    image = images[idx]
#     filename = filenames[idx]

    #txtfile = open(savd_dir+)
    
    boxes = annotations[idx][0]
    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)
    
    seq = iaa.Sequential([
        iaa.Snowflakes(flake_size=(0.1, 0.4), speed=(0.01, 0.05))
        #iaa.Snowflakes(flake_size=(0.2, 0.7), speed=(0.007, 0.03))
    ], random_order=True)
    
    seq_det = seq.to_deterministic()

    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    
    
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            i,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )

    
    
    new_image_file = save_dir + 'snow_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    new_txt_file = open(save_dir+'snow_'+annotations[idx][1], 'w')

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        bb_box_ = [int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2)]
        yolo_style = yolo_convert((w,h), bb_box_)
        line = ("%s %f %f %f %f\n" % (boxes[i][0],yolo_style[0], yolo_style[1],yolo_style[2], yolo_style[3]))
        new_txt_file.write(line)

    
    new_txt_file.close()
    

print("done!!")



BB 0: (105.0000, 121.0000, 129.0000, 145.0000) -> (105.0000, 121.0000, 129.0000, 145.0000)
BB 1: (80.0000, 97.0000, 147.0000, 155.0000) -> (80.0000, 97.0000, 147.0000, 155.0000)
BB 0: (155.0000, 97.0000, 355.0000, 145.0000) -> (155.0000, 97.0000, 355.0000, 145.0000)
BB 1: (351.0000, 131.0000, 461.0000, 327.0000) -> (351.0000, 131.0000, 461.0000, 327.0000)
BB 0: (142.0000, 77.0000, 157.0000, 113.0000) -> (142.0000, 77.0000, 157.0000, 113.0000)
BB 0: (96.0000, 100.0000, 138.0000, 123.0000) -> (96.0000, 100.0000, 138.0000, 123.0000)
BB 0: (286.0000, 99.0000, 401.0000, 119.0000) -> (286.0000, 99.0000, 401.0000, 119.0000)
BB 1: (407.0000, 114.0000, 515.0000, 180.0000) -> (407.0000, 114.0000, 515.0000, 180.0000)
BB 2: (579.0000, 291.0000, 628.0000, 390.0000) -> (579.0000, 291.0000, 628.0000, 390.0000)
BB 0: (72.0000, 45.0000, 193.0000, 124.0000) -> (72.0000, 45.0000, 193.0000, 124.0000)
BB 1: (167.0000, 27.0000, 271.0000, 70.0000) -> (167.0000, 27.0000, 271.0000, 70.0000)
BB 2: (19.0000, 109

BB 0: (85.0000, 84.0000, 491.0000, 254.0000) -> (85.0000, 84.0000, 491.0000, 254.0000)
BB 0: (192.0000, 218.0000, 282.0000, 296.0000) -> (192.0000, 218.0000, 282.0000, 296.0000)
BB 0: (128.0000, 217.0000, 182.0000, 348.0000) -> (128.0000, 217.0000, 182.0000, 348.0000)
BB 0: (62.0000, 84.0000, 161.0000, 99.0000) -> (62.0000, 84.0000, 161.0000, 99.0000)
BB 1: (1.0000, 92.0000, 61.0000, 104.0000) -> (1.0000, 92.0000, 61.0000, 104.0000)
BB 0: (111.0000, 62.0000, 208.0000, 115.0000) -> (111.0000, 62.0000, 208.0000, 115.0000)
BB 0: (258.0000, 119.0000, 418.0000, 248.0000) -> (258.0000, 119.0000, 418.0000, 248.0000)
BB 0: (150.0000, 129.0000, 277.0000, 214.0000) -> (150.0000, 129.0000, 277.0000, 214.0000)
BB 0: (75.0000, 56.0000, 103.0000, 90.0000) -> (75.0000, 56.0000, 103.0000, 90.0000)
BB 0: (173.0000, 64.0000, 193.0000, 82.0000) -> (173.0000, 64.0000, 193.0000, 82.0000)
BB 0: (71.0000, 173.0000, 134.0000, 229.0000) -> (71.0000, 173.0000, 134.0000, 229.0000)
BB 0: (50.0000, 40.0000, 259.00

BB 0: (230.0000, 155.0000, 272.0000, 194.0000) -> (230.0000, 155.0000, 272.0000, 194.0000)
BB 1: (146.0000, 157.0000, 353.0000, 196.0000) -> (146.0000, 157.0000, 353.0000, 196.0000)
BB 2: (9.0000, 115.0000, 101.0000, 160.0000) -> (9.0000, 115.0000, 101.0000, 160.0000)
BB 3: (104.0000, 151.0000, 215.0000, 191.0000) -> (104.0000, 151.0000, 215.0000, 191.0000)
BB 4: (294.0000, 172.0000, 377.0000, 199.0000) -> (294.0000, 172.0000, 377.0000, 199.0000)
BB 0: (138.0000, 175.0000, 235.0000, 265.0000) -> (138.0000, 175.0000, 235.0000, 265.0000)
BB 0: (61.0000, 68.0000, 78.0000, 89.0000) -> (61.0000, 68.0000, 78.0000, 89.0000)
BB 1: (74.0000, 92.0000, 89.0000, 107.0000) -> (74.0000, 92.0000, 89.0000, 107.0000)
BB 0: (180.0000, 111.0000, 274.0000, 211.0000) -> (180.0000, 111.0000, 274.0000, 211.0000)
BB 0: (188.0000, 132.0000, 227.0000, 174.0000) -> (188.0000, 132.0000, 227.0000, 174.0000)
BB 0: (74.0000, 93.0000, 151.0000, 105.0000) -> (74.0000, 93.0000, 151.0000, 105.0000)
BB 1: (140.0000, 88.0

BB 0: (107.0000, 67.0000, 204.0000, 95.0000) -> (107.0000, 67.0000, 204.0000, 95.0000)
BB 0: (1.0000, 54.0000, 153.0000, 89.0000) -> (1.0000, 54.0000, 153.0000, 89.0000)
BB 0: (246.0000, 163.0000, 576.0000, 225.0000) -> (246.0000, 163.0000, 576.0000, 225.0000)
BB 0: (129.0000, 142.0000, 217.0000, 255.0000) -> (129.0000, 142.0000, 217.0000, 255.0000)
BB 0: (281.0000, 180.0000, 368.0000, 219.0000) -> (281.0000, 180.0000, 368.0000, 219.0000)
BB 0: (46.0000, 45.0000, 178.0000, 136.0000) -> (46.0000, 45.0000, 178.0000, 136.0000)
BB 0: (35.0000, 90.0000, 198.0000, 145.0000) -> (35.0000, 90.0000, 198.0000, 145.0000)
BB 0: (3.0000, 90.0000, 153.0000, 147.0000) -> (3.0000, 90.0000, 153.0000, 147.0000)
BB 1: (45.0000, 130.0000, 157.0000, 205.0000) -> (45.0000, 130.0000, 157.0000, 205.0000)
BB 0: (191.0000, 103.0000, 226.0000, 217.0000) -> (191.0000, 103.0000, 226.0000, 217.0000)
BB 0: (62.0000, 6.0000, 84.0000, 120.0000) -> (62.0000, 6.0000, 84.0000, 120.0000)
BB 0: (241.0000, 173.0000, 341.0000

BB 0: (71.0000, 6.0000, 164.0000, 95.0000) -> (71.0000, 6.0000, 164.0000, 95.0000)
BB 0: (210.0000, 187.0000, 250.0000, 221.0000) -> (210.0000, 187.0000, 250.0000, 221.0000)
BB 1: (189.0000, 226.0000, 246.0000, 254.0000) -> (189.0000, 226.0000, 246.0000, 254.0000)
BB 0: (21.0000, 100.0000, 87.0000, 169.0000) -> (21.0000, 100.0000, 87.0000, 169.0000)
BB 1: (24.0000, 71.0000, 83.0000, 170.0000) -> (24.0000, 71.0000, 83.0000, 170.0000)
BB 0: (38.0000, 53.0000, 89.0000, 124.0000) -> (38.0000, 53.0000, 89.0000, 124.0000)
BB 1: (42.0000, 163.0000, 87.0000, 219.0000) -> (42.0000, 163.0000, 87.0000, 219.0000)
BB 0: (62.0000, 61.0000, 85.0000, 205.0000) -> (62.0000, 61.0000, 85.0000, 205.0000)
BB 0: (108.0000, 31.0000, 162.0000, 132.0000) -> (108.0000, 31.0000, 162.0000, 132.0000)
BB 0: (86.0000, 69.0000, 120.0000, 98.0000) -> (86.0000, 69.0000, 120.0000, 98.0000)
BB 0: (93.0000, 85.0000, 108.0000, 258.0000) -> (93.0000, 85.0000, 108.0000, 258.0000)
BB 1: (79.0000, 114.0000, 88.0000, 152.0000) 

BB 0: (146.0000, 105.0000, 304.0000, 237.0000) -> (146.0000, 105.0000, 304.0000, 237.0000)
BB 1: (102.0000, 74.0000, 324.0000, 308.0000) -> (102.0000, 74.0000, 324.0000, 308.0000)
BB 0: (92.0000, 87.0000, 173.0000, 147.0000) -> (92.0000, 87.0000, 173.0000, 147.0000)
BB 0: (179.0000, 195.0000, 248.0000, 251.0000) -> (179.0000, 195.0000, 248.0000, 251.0000)
BB 0: (89.0000, 214.0000, 142.0000, 271.0000) -> (89.0000, 214.0000, 142.0000, 271.0000)
BB 0: (173.0000, 127.0000, 247.0000, 273.0000) -> (173.0000, 127.0000, 247.0000, 273.0000)
BB 1: (249.0000, 131.0000, 316.0000, 255.0000) -> (249.0000, 131.0000, 316.0000, 255.0000)
BB 0: (151.0000, 136.0000, 241.0000, 241.0000) -> (151.0000, 136.0000, 241.0000, 241.0000)
BB 0: (35.0000, 51.0000, 88.0000, 114.0000) -> (35.0000, 51.0000, 88.0000, 114.0000)
BB 0: (71.0000, 98.0000, 113.0000, 149.0000) -> (71.0000, 98.0000, 113.0000, 149.0000)
BB 1: (60.0000, 146.0000, 96.0000, 182.0000) -> (60.0000, 146.0000, 96.0000, 182.0000)
BB 0: (60.0000, 107.0

In [13]:
import xml.etree.ElementTree as ET
import imgaug as ia
from imgaug import augmenters as iaa
from files import *
from os import listdir
import cv2
import numpy as np
from pascal_voc_writer import Writer

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower() or 'jpeg' in file.lower() or 'JPEG' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations



ia.seed(1)

img_dir = '/home/capstone7/darkflow/data/yolo_new/'
save_dir = '/home/capstone7/darkflow/data/yolo_all_aug/'
images, annotations = read_train_dataset(img_dir)
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

for idx in range(len(images)):
    image = images[idx]
    boxes = annotations[idx][0]

    ia_bounding_boxes = []
    for box in boxes:
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

    seq = iaa.Sequential([
        iaa.Sometimes(0.7, iaa.Add(-90)),
        iaa.Sometimes(0.7,iaa.Multiply((0.5, 1.25))),
        iaa.Sometimes(0.5,iaa.MotionBlur(k=5, angle=[-45,45])),
        iaa.Sometimes(0.5, iaa.Fliplr(0.5))
    ], random_order=True)

    seq_det = seq.to_deterministic()

    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

    new_image_file = save_dir + 'aug_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    voc_writer = Writer(new_image_file, w, h)

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

    voc_writer.save(save_dir + 'aug_' + annotations[idx][1])

print("done")

done
